In [1]:
!pip install transformers
!pip install miditok
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 6.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import tensorflow as tf
import transformers
import miditok
import numpy
import os
import pathlib
from tensorflow import keras
import json
from transformers import TFGPT2LMHeadModel, GPT2Config, Trainer, TrainingArguments, GenerationConfig
from miditok import MIDILike, MIDITokenizer
from miditok.constants import CHORD_MAPS
from miditoolkit import MidiFile
import tqdm

In [3]:
attn_window = 10
j = 0
input_seq = [] 
target_seq = []
batch_size = 16
buffer_size = 1000


In [4]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [5]:
directory = "/content/gdrive/MyDrive/tokens_BPE"

for fname in os.listdir(directory):                     # for each file in the directory
    with open(os.path.join(directory, fname)) as i:     # open the file
        data = json.load(i)                             # load the json file                             

    for i in range(0, len(data['ids'][0])-1, 11):    # for each token in the file
            x = data['ids'][0][i:i+attn_window]      # get the next 10 tokens
            y = data['ids'][0][i+1:i+attn_window+1]  # get the next 1+10 tokens

            if len(x) == attn_window and len(y) == attn_window: # if the length of the tokens is 10
                
                print('batch number:', j)            # print the batch number
                print('input:', x)                   
                print('target:', y) 
                print('-----')

                j += 1                               # increment the batch number

                input_seq.append(x)                  # append the input sequence
                target_seq.append(y)                 # append the target sequence

Streaming output truncated to the last 5000 lines.
batch number: 135577
input: [209, 62, 209, 74, 209, 218, 150, 66, 209, 214]
target: [62, 209, 74, 209, 218, 150, 66, 209, 214, 123]
-----
batch number: 135578
input: [135, 142, 162, 154, 38, 209, 50, 209, 57, 209]
target: [142, 162, 154, 38, 209, 50, 209, 57, 209, 64]
-----
batch number: 135579
input: [209, 69, 209, 218, 71, 209, 214, 482, 145, 152]
target: [69, 209, 218, 71, 209, 214, 482, 145, 152, 157]
-----
batch number: 135580
input: [159, 45, 209, 52, 209, 61, 209, 66, 209, 222]
target: [45, 209, 52, 209, 61, 209, 66, 209, 222, 140]
-----
batch number: 135581
input: [149, 154, 214, 69, 209, 218, 133, 157, 43, 209]
target: [154, 214, 69, 209, 218, 133, 157, 43, 209, 50]
-----
batch number: 135582
input: [209, 55, 209, 62, 209, 67, 209, 218, 155, 66]
target: [55, 209, 62, 209, 67, 209, 218, 155, 66, 209]
-----
batch number: 135583
input: [329, 47, 209, 59, 209, 66, 209, 74, 209, 218]
target: [47, 209, 59, 209, 66, 209, 74, 209, 218

In [6]:
print('input_seq length:', len(input_seq))
print('target_seq length:', len(target_seq))

input_seq length: 136827
target_seq length: 136827


In [7]:
from sklearn.model_selection import train_test_split

train_size = 0.8 # 80/20 split between train and test+val data
test_size = 0.5 # 50/50 (80/10/10) split between train, test and val

x_train, x_rem, y_train, y_rem = train_test_split(input_seq, target_seq, train_size = 0.8)
x_test, x_val, y_test, y_val = train_test_split(x_rem, y_rem, test_size = 0.5)


In [8]:
print('train input:', len(x_train)), print('train target:', len(y_train))
print('val input:', len(x_val)), print('val target:', len(y_val))
print('test input:', len(x_test)), print('test target:', len(y_test))

train input: 109461
train target: 109461
val input: 13683
val target: 13683
test input: 13683
test target: 13683


(None, None)

In [9]:
pitch_range = range(21, 109)
beat_res = {(0, 4): 8, (4, 12): 4}
nb_velocities = 32
additional_tokens = {'Chord': True, 'Rest': True, 'Tempo': True,
                     'rest_range': (2, 8),  # (half, 8 beats)
                     'nb_tempos': 32,  # nb of tempo bins
                     'tempo_range': (40, 250),  # (min, max)
                     'Program': False,
                     "chord_maps": CHORD_MAPS,
                     "chord_tokens_with_root_note": True,
                     "chord_unknown": False}
special_tokens = ["PAD", "BOS", "EOS"]

In [10]:
#creates tokenizer
tokenizer = MIDILike(pitch_range, beat_res, nb_velocities, additional_tokens, special_tokens=special_tokens)

In [11]:
# Creates model
config = GPT2Config(
    vocab_size=500,
    n_positions=2048,
    n_embd=512,
    n_layer=8,
    n_head=8,
    n_inner=2048,
    resid_pdrop=.1,
    embd_pdrop=.1,
    attn_pdrop=.1,
    padding_token_id=tokenizer['PAD_None'],
    bos_token_id=tokenizer['BOS_None'],
    eos_token_id=tokenizer['EOS_None'],
)
model = TFGPT2LMHeadModel(config)

In [12]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# defining our metric which we want to observe
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# compiling the model
model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])


In [ ]:
num_epoch = 15

# Create a callback that saves the model's weights
cp_callback = [
tf.keras.callbacks.ModelCheckpoint("/content/gdrive/MyDrive/models/ala_model.keras",
                                                 save_weights_only=True,
                                                 verbose=1)
]

history = model.fit(x_train, y_train, epochs=num_epoch, callbacks=[cp_callback], validation_data = (x_val, y_val))

Epoch 1/15
3421/3421 [==============================] - ETA: 0s - loss: 3.6395 - accuracy: 0.2532
Epoch 1: saving model to /content/gdrive/MyDrive/models/ala_model.keras
3421/3421 [==============================] - 5624s 2s/step - loss: 3.6395 - accuracy: 0.2532 - val_loss: 2.9278 - val_accuracy: 0.3366
Epoch 2/15
3421/3421 [==============================] - ETA: 0s - loss: 2.8748 - accuracy: 0.3420
Epoch 2: saving model to /content/gdrive/MyDrive/models/ala_model.keras
3421/3421 [==============================] - 5614s 2s/step - loss: 2.8748 - accuracy: 0.3420 - val_loss: 2.7143 - val_accuracy: 0.3685
Epoch 3/15
3421/3421 [==============================] - ETA: 0s - loss: 2.7040 - accuracy: 0.3676
Epoch 3: saving model to /content/gdrive/MyDrive/models/ala_model.keras
3421/3421 [==============================] - 5638s 2s/step - loss: 2.7040 - accuracy: 0.3676 - val_loss: 2.5991 - val_accuracy: 0.3878
Epoch 4/15
3421/3421 [==============================] - ETA: 0s - loss: 2.5918 - accu